In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
nltk.download("stopwords")
nltk.download('punkt')
stop_words= set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


function to remove stopwords

In [4]:
def remove_stopwods(df,label):
  j=0
  for i in df[label]:
    text_tokens=word_tokenize(i)
    filtered_words=[word for word in text_tokens if word not in stop_words]
    df[label][j]=filtered_words
    j=j+1

Importing DataSet of news

In [5]:
  # #Arnav Directory
  # from google.colab import drive 
  # drive.mount("/content/gdrive")
  # data = pd.read_csv("/content/gdrive/My Drive/JioMeetSentiments-2.csv")
  # test_data=pd.read_csv("/content/gdrive/My Drive/NewsData.csv")

In [6]:
 # ## rajat directory
from google.colab import drive 
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [7]:
data = pd.read_csv("/content/gdrive/My Drive/jio_data/JioMeetSentiments.csv")

In [8]:
data=data[["sentimenttype","simplifiedtext"]]
data

,sentimenttype,simplifiedtext
0,2,SUPER APP
1,1,PL ALLOW REGULAR MEETINGS WITH SAME LINK AND ...
2,0,PROVIDE WHITE BOARD
3,2,THIS IS AN BEST APP I LIKE IT
4,2,EASY TO USE AND VERY SMART APP
...,...,...
23722,2,MERA JIO MEET ON KU NHI HO RHA
23723,2,AWESOME JUST LOVE IT WAY BETTER THAN OTHER E...
23724,0,NO S� PUEDE CAMBIAR EL IDIOMA A ESPA�OL
23725,0,IT S TOO GOOD BUT I REQUEST YOU TO MAKE UPTIL...


In [9]:
data['simplifiedtext']= data['simplifiedtext'].astype("str")

In [10]:
data['sentimenttype']=data['sentimenttype'].fillna(1)
data['sentimenttype']=data['sentimenttype'].astype("int")

In [11]:
remove_stopwods(data,"simplifiedtext")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
data.head()

,sentimenttype,simplifiedtext
0,2,"[SUPER, APP]"
1,1,"[PL, ALLOW, REGULAR, MEETINGS, WITH, SAME, LIN..."
2,0,"[PROVIDE, WHITE, BOARD]"
3,2,"[THIS, IS, AN, BEST, APP, I, LIKE, IT]"
4,2,"[EASY, TO, USE, AND, VERY, SMART, APP]"


In [13]:
def convertList(list1):  
    str = ''  # initializing the empty string  
    for i in list1: #Iterating and adding the list element to the str variable  
      str =str+ " "+i
    return str

In [14]:
k=0
while(k<len(data['simplifiedtext'])):
  data['simplifiedtext'][k]= convertList(data['simplifiedtext'][k])
  k=k+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
data['simplifiedtext'] = data['simplifiedtext'].apply(lambda x: x.lower())
data['simplifiedtext'] = data['simplifiedtext'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


max_fatures = 25000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') # giving unique id to each word
tokenizer.fit_on_texts(data['simplifiedtext'].values)
X = tokenizer.texts_to_sequences(data['simplifiedtext'].values)
X = pad_sequences(X)

In [16]:
embed_dim = 128  
lstm_out = 196
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 111, 128)          3200000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 111, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 3)                 591       
Total params: 3,455,391
Trainable params: 3,455,391
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
Y = pd.get_dummies(data['sentimenttype']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

In [18]:
batch_size = 64
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
297/297 - 266s - loss: 0.4905 - accuracy: 0.8021
Epoch 2/5
297/297 - 237s - loss: 0.2730 - accuracy: 0.9027
Epoch 3/5
297/297 - 237s - loss: 0.2048 - accuracy: 0.9308
Epoch 4/5
297/297 - 236s - loss: 0.1603 - accuracy: 0.9463
Epoch 5/5
297/297 - 235s - loss: 0.1340 - accuracy: 0.9584


Extracting a validation set, and measuring score and accuracy.

## Finding accuracy of the model using metrics


In [19]:
pred1=model.predict(X_test)
pred2=list()
for i in pred1:
 pred2.append(np.argmax(i))
pred2=np.array(pred2)
pred2 

array([0, 0, 0, ..., 2, 1, 1])

Converting y_test to again its label for finding accuracy

In [20]:
Y_test1=[]
for i in Y_test:
  if np.argmax(i)==0:
    Y_test1.append(0)
  elif np.argmax(i)==1:
    Y_test1.append(1)
  else :
    Y_test1.append(2)
Y_test1=np.array(Y_test1)
Y_test1 

array([0, 0, 0, ..., 2, 1, 1])

In [21]:
from sklearn.metrics import f1_score
f1=f1_score(Y_test1,pred2,average='weighted')
f1

0.9100907671724014

In [24]:
pred2[0:200]

array([0, 0, 0, 1, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 1, 2,
       0, 1, 0, 0, 2, 0, 2, 2, 0, 0, 0, 1, 1, 1, 2, 1, 2, 0, 0, 1, 0, 0,
       2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2,
       2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 1, 0, 2, 0,
       1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2,
       2, 2, 2, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2,
       2, 0, 0, 2, 2, 0, 2, 0, 2, 1, 1, 1, 0, 2, 0, 2, 0, 2, 0, 2, 2, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2,
       2, 0])